# Simulation of Local Protocols

This notebook is meant to be used to simulate a local protocol supplemented by one bit of communication. The protocol is simulated using Monte-Carlo simulation.

In [1]:
# Import the required packages
import numpy as np
import math
from numpy.random import default_rng
rng = default_rng()

First, let us define some functions

In [2]:
def random_vector(n):
    """Generate an uniformly distributed random vector."""
    components = [rng.standard_normal() for i in range(n)]
    r = math.sqrt(sum(x*x for x in components))
    v = np.array([x/r for x in components])
    return v


def dot(vec_1, vec_2):
    """Dot product for two vectors."""
    return vec_1[0] * vec_2[0] + vec_1[1] * vec_2[1] + vec_1[2] * vec_2[2]


def outer_product(vec_1, vec_2):
    """Do an outer product then flatten the vector. Used for mixing the probabilities."""
    return np.outer(vec_1, vec_2).flatten()

Now we define the functions that act as the protocols

In [3]:
def alice_protocol_1(vec_alice, lhv):
    """First protocol of Alice. Output is in the form of (P(A=+1), P(A=-1))"""
    prob = 1/2 - 1/2 * np.sign(dot(vec_alice, lhv[0]))
    return np.array([prob, 1-prob])


def alice_protocol_2(vec_alice, lhv):
    """Second protocol of Alice. Output is in the form of (P(A=+1), P(A=-1))"""
    return alice_protocol_1(vec_alice, lhv)


def bob_protocol_1(vec_bob, lhv):
    """First protocol of Bob. Output is in the form of (P(A=+1), P(A=-1))"""
    prob = 1/2 + 1/2 * np.sign(dot(vec_bob, lhv[0] - lhv[1]))
    return np.array([prob, 1-prob])


def bob_protocol_2(vec_bob, lhv):
    """Second protocol of Bob. Output is in the form of (P(A=+1), P(A=-1))"""
    prob = 1/2 + 1/2 * np.sign(dot(vec_bob, lhv[0] + lhv[1]))
    return np.array([prob, 1-prob])


def comm_protocol(vec_alice, lhv):
    """Protocol for the communication bit. The output is the probability of choosing the first protocols."""
    return 1/2 - 1/2 * np.sign(dot(vec_alice, lhv[0])) * np.sign(dot(vec_alice, lhv[1]))

The following codes are to show how the outer product works

In [4]:
alice = np.array([0.1, 0.9]) # -> P(+1), P(-1)
bob = np.array([0.3, 0.7]) # -> P(+1), P(-1)

print(outer_product(alice, bob))
# output is in the form of P(AB) -> (P(+1+1), P(+1-1), P(-1+1), P(-1-1))

[0.03 0.07 0.27 0.63]


Now we define the functions that calculate the probability from the protocols

In [5]:
def probability_single_round(vec_alice, vec_bob, lhv):
    """Calculate the probability distribution from one round of LHV."""
    bit = comm_protocol(vec_alice, lhv)
    return bit * outer_product(alice_protocol_1(vec_alice, lhv), bob_protocol_1(vec_bob, lhv)) + \
        (1 - bit) * outer_product(alice_protocol_2(vec_alice, lhv), bob_protocol_2(vec_bob, lhv))
    
    
def probability(vec_alice, vec_bob, lhv_type="double-vector", n=100000):
    """Calculate the probability distribution from multiple rounds of LHV."""
    prob = np.array([0.0, 0.0, 0.0, 0.0])
    for i in range(n):
        if lhv_type == "single-vector":
            lhv = random_vector(3)
        elif lhv_type == "double-vector":
            lhv = (random_vector(3), random_vector(3))
        prob += probability_single_round(vec_alice, vec_bob, lhv)
    return prob / n

Now let us define a function to evaluate the expectations.

In [6]:
def expectations(vec_alice, vec_bob, lhv_type="double-vector", n=100000):
    """Print the expectations of the protocols."""
    prob = probability(vec_alice, vec_bob, lhv_type=lhv_type, n=n)
    print("Expectation of Alice :", prob[0] + prob[1] - prob[2] - prob[3])
    print("Expectation of Bob   :", prob[0] - prob[1] + prob[2] - prob[3])
    print("Joint expectation    :", prob[0] - prob[1] - prob[2] + prob[3])

In [7]:
vec_alice, vec_bob = random_vector(3), random_vector(3)
print(dot(vec_alice, vec_bob))
expectations(vec_alice, vec_bob)

-0.3002530041748922
Expectation of Alice : 0.00024000000000001798
Expectation of Bob   : 0.0036199999999999566
Joint expectation    : 0.30177999999999994


In [8]:
a = np.array([-0.3562,-0.7127,0.6043])
b = np.array([-0.7403,0.0806,-0.6674])
expectations(a, b, n=1000000)
print(dot(a, b))

Expectation of Alice : -0.0007339999999999569
Expectation of Bob   : 2.399999999996849e-05
Joint expectation    : 0.19470199999999996
-0.19705858


In [9]:
def normalise(vector):
    r = math.sqrt(sum(x*x for x in vector))
    v = np.array([x/r for x in vector])
    return v

def rotation(axis, angle):
    axis = normalise(axis)
    u1, u2, u3 = axis
    r11 = np.cos(angle) + u1 ** 2 * (1 - np.cos(angle))
    r12 = u1 * u2 * (1 - np.cos(angle)) - u3 * np.sin(angle)
    r13 = u1 * u3 * (1 - np.cos(angle)) + u2 * np.sin(angle)
    r21 = u2 * u1 * (1 - np.cos(angle)) + u3 * np.sin(angle)
    r22 = np.cos(angle) + u2 ** 2 * (1 - np.cos(angle))
    r23 = u2 * u3 * (1 - np.cos(angle)) - u1 * np.sin(angle)
    r31 = u3 * u1 * (1 - np.cos(angle)) - u2 * np.sin(angle)
    r32 = u3 * u2 * (1 - np.cos(angle)) + u1 * np.sin(angle)
    r33 = np.cos(angle) + u3 ** 2 * (1 - np.cos(angle))
    return np.array([[r11, r12, r13], [r21, r22, r23], [r31, r32, r33]])